todo organize topcs within datasets and numofentries

save all new configs in drive

requirements

In [0]:
!pip install tensorflowjs

* Business Domain
* Medical
Medical Standards
Clinical Trials
* Pharma
* Law
* Finance
* Insurance
* Marketing
* Advertisement
* Manufacturing
* Renewables & Environment


### load data

In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/data/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
features = np.load('gdrive/My Drive/data/train_test_data/features.npy').item()
vectors_test = np.load('gdrive/My Drive/data/train_test_data/vectors_test.npy').item()
# bin_tes_y = np.genfromtxt('bin_tes_y', delimiter=',')
# bin_tr_y = np.genfromtxt('bin_tr_y', delimiter=',')


y_test = pd.read_csv('gdrive/My Drive/data/train_test_data/unprocessed/y_test', usecols=[1], header=None)
y_train = pd.read_csv('gdrive/My Drive/data/train_test_data/unprocessed/y_train', usecols=[1], header=None)
# y_test = np.genfromtxt('y_test', delimiter=',')[:,1]
# y_train = np.genfromtxt('y_train', delimiter=',')[:,1]
# x_train = np.genfromtxt('x_train', delimiter=',')
# x_test = np.genfromtxt('x_test', delimiter=',')

In [0]:
y_train.shape, y_test.shape, features.shape, vectors_test.shape

((83761, 1), (27921, 1), (83761, 70436), (27921, 70436))

In [0]:
chosen = [1,2,3,4,5,11,12,13,14,20,24,25]

train_mask = np.isin(y_train, chosen).flatten() 
test_mask = np.isin(y_test, chosen).flatten()

y_test = y_test[test_mask].replace({1:0, 2:0, 3:0, 4:0, 5:0, 11:1, 12:2, 13:3, 14:4, 20:5, 24:6, 25:7})
y_train = y_train[train_mask].replace({1:0, 2:0, 3:0, 4:0, 5:0, 11:1, 12:2, 13:3, 14:4, 20:5, 24:6, 25:7})
features = features[train_mask]
vectors_test = vectors_test[test_mask]

In [0]:
y_train.shape, y_test.shape, features.shape, vectors_test.shape

((45071, 1), (15023, 1), (45071, 70436), (15023, 70436))

In [0]:
from tensorflow.keras.utils import to_categorical
y_binary_train = to_categorical(y_train)
y_binary_test = to_categorical(y_test)

### Model & train

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [0]:
simple_model = Sequential([Dense(200,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.001)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.001)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(50,activation='relu', kernel_regularizer=l2(0.001)),
                           Dropout(0.1, noise_shape=None, seed=None),
                           Dense(6,activation='sigmoid')])

ES_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=4)

filepath="improvement-{epoch:02d}-{loss:.4f}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

simple_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

model_output= simple_model.fit(features,
                               y_binary_train,
                               batch_size=128,
                               epochs=40,
                               verbose=1,
                               validation_data=(vectors_test, y_binary_test),
                               callbacks=[ES_callback, checkpoint])
                              

In [0]:
simple_model = Sequential([Dense(200,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.001)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.001)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(50,activation='relu', kernel_regularizer=l2(0.001)),
                           Dropout(0.1, noise_shape=None, seed=None),
                           Dense(6,activation='sigmoid')])

simple_model.load_weights('improvement-08-1.2047-0.8255.hdf5')

simple_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

### Model evaluation


In [0]:
predicted = simple_model.predict_classes(vectors_test)
predicted

In [0]:
confusion_matrix(y_test, predicted)

array([[1978,   69,  156,  285,   37,   89],
       [  71, 2129,  110,  229,   57,   25],
       [  94,  761, 1617,  119,   11,   43],
       [ 123,  133,   48, 9389,  296,  179],
       [  19,   45,   13,  457, 1908,   75],
       [  75,   40,   38,  233,   98, 2033]])

In [0]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.84      0.76      0.80      2614
           1       0.67      0.81      0.73      2621
           2       0.82      0.61      0.70      2645
           3       0.88      0.92      0.90     10168
           4       0.79      0.76      0.77      2517
           5       0.83      0.81      0.82      2517

   micro avg       0.83      0.83      0.83     23082
   macro avg       0.80      0.78      0.79     23082
weighted avg       0.83      0.83      0.82     23082



### Load model

In [0]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(simple_model, 'model')